In [1]:
!pip install transformers datasets pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv('/content/ielts_writing_dataset (1).csv')
df.head()

,Task_Type,Question,Essay,Examiner_Commen,Task_Response,Coherence_Cohesion,Lexical_Resource,Range_Accuracy,Overall
0,1,The bar chart below describes some changes abo...,"Between 1995 and 2010, a study was conducted r...",NaN,NaN,NaN,NaN,NaN,5.5
1,2,Rich countries often give money to poorer coun...,Poverty represents a worldwide crisis. It is t...,NaN,NaN,NaN,NaN,NaN,6.5
2,1,The bar chart below describes some changes abo...,The left chart shows the population change hap...,NaN,NaN,NaN,NaN,NaN,5.0
3,2,Rich countries often give money to poorer coun...,Human beings are facing many challenges nowada...,NaN,NaN,NaN,NaN,NaN,5.5
4,1,The graph below shows the number of overseas v...,Information about the thousands of visits from...,NaN,NaN,NaN,NaN,NaN,7.0


In [7]:
df = df[["Essay", "Overall"]]
df.head()

,Essay,Overall
0,"Between 1995 and 2010, a study was conducted r...",5.5
1,Poverty represents a worldwide crisis. It is t...,6.5
2,The left chart shows the population change hap...,5.0
3,Human beings are facing many challenges nowada...,5.5
4,Information about the thousands of visits from...,7.0


In [8]:
df_test = df.sample(n = 100, random_state = 42)
df_test.head()

,Essay,Overall
752,The graphic illustrates the nutritional consis...,6.5
857,Life is full of competition. Soon after the re...,7.5
629,Sugar-based drinks has become a popular bevera...,7.0
1411,"In the Third World, children are usually sent ...",6.0
974,The line graph illustrates the proportion of m...,5.5


In [9]:
df = df[~df.index.isin(df_test.index)]
df.shape

(1335, 2)

In [10]:
dataset = Dataset.from_pandas(df, preserve_index = False)
dataset = dataset.train_test_split(test_size=0.1)

In [15]:
import transformers
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=1)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Essay', 'Overall'],
        num_rows: 1201
    })
    test: Dataset({
        features: ['Essay', 'Overall'],
        num_rows: 134
    })
})

In [42]:
def tokenize_function(examples):
    label = examples["Overall"]
    examples = tokenizer(examples["Essay"], truncation=True, padding="max_length", max_length=256)

    examples["label"] = float(label)
    return examples

for split in dataset:
    dataset[split] = dataset[split].map(tokenize_function, remove_columns=["Essay", "Overall"])

Map:   0%|          | 0/1201 [00:00<?, ? examples/s]

Map:   0%|          | 0/134 [00:00<?, ? examples/s]

In [24]:
#tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'Overall'])

In [27]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.3 MB/s eta 0:00:00


In [34]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

def compute_metrics_for_regression(eval_pred):
    logits, labels = eval_pred
    labels = labels.reshape(-1, 1)

    mse = mean_squared_error(labels, logits)
    mae = mean_absolute_error(labels, logits)
    r2 = r2_score(labels, logits)
    single_squared_errors = ((logits - labels).flatten()**2).tolist()
    accuracy = sum([1 for e in single_squared_errors if e < 0.25]) / len(single_squared_errors)

    return {"mse": mse, "mae": mae, "r2": r2, "accuracy": accuracy}

In [ ]:
from transformers import TrainingArguments, Trainer, TrainerCallback
import torch

training_args = TrainingArguments(output_dir="./fine_tuned_bert",
                                  logging_strategy="epoch",
                                  evaluation_strategy="epoch",
                                  learning_rate= 8e-5,
                                  per_device_train_batch_size=32,
                                  per_device_eval_batch_size=32,
                                  num_train_epochs=20,
                                  save_total_limit = 2,
                                  save_strategy = 'epoch',
                                  lr_scheduler_type = "linear",
                                  optim = "adamw_torch",
                                  weight_decay = 0.01,
                                  load_best_model_at_end=True,
                                  metric_for_best_model="mae",
                                  warmup_steps=500,
                                  )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics_for_regression
)

class MetricsCallback(TrainerCallback):
    def __init__(self):
        self.training_history = {"train": [], "eval": []}

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            if "loss" in logs:  # Training logs
                self.training_history["train"].append(logs)
            elif "eval_loss" in logs:  # Evaluation logs
                self.training_history["eval"].append(logs)

metrics_callback = MetricsCallback()
trainer.add_callback(metrics_callback)

trainer.train()

Epoch,Training Loss,Validation Loss,Mse,Mae,R2,Accuracy
1,0.082700,0.534766,0.534766,0.559056,0.424265,0.582090
2,0.077600,0.517248,0.517248,0.543320,0.443124,0.597015
3,0.088400,0.589480,0.589480,0.593169,0.365359,0.552239
4,0.091000,0.714591,0.714591,0.667290,0.230663,0.492537
5,0.071500,0.596322,0.596322,0.591154,0.357992,0.559701
6,0.085000,0.605287,0.605287,0.608228,0.348341,0.559701
7,0.110900,0.662609,0.662609,0.628911,0.286627,0.507463
8,0.076800,0.663275,0.663275,0.634133,0.285910,0.514925
9,0.106300,0.553704,0.553704,0.563025,0.403875,0.574627
10,0.115300,0.610445,0.610445,0.608134,0.342787,0.529851


In [48]:
dataset_2 = Dataset.from_pandas(df_test, preserve_index = False)
dataset_2 = dataset_2.map(tokenize_function, remove_columns=["Essay", "Overall"])

trainer.eval_dataset=dataset_2
trainer.evaluate()


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

{'eval_loss': 0.7224123477935791,
 'eval_mse': 0.7224123477935791,
 'eval_mae': 0.6385540962219238,
 'eval_r2': 0.39036935567855835,
 'eval_accuracy': 0.47,
 'eval_runtime': 0.7589,
 'eval_samples_per_second': 131.778,
 'eval_steps_per_second': 5.271,
 'epoch': 10.0}